# Calculating Visibility in Counter-Strike 2

Calculating visibility is a bit difficult for a variety of reasons. You may want to use the `isSpotted` flag that has long been in Counter-Strike demos. However, it is known to be unreliable. We can flip the problem of visibility by doing some basic geometrical calculations. At the simplest level, we can see if a line segment between two points intersects any part of the map. Now, this is clearly not 100% foolproof, as smokes can obscure vision or a player's head position can even change visibility, but it is a start.

First, you need to acquire the Universal Scene Description (USD) files by running `awpy get usd`. You can see how we create USDs by visiting [here](https://awpy.readthedocs.io/en/latest/usd.html). The interface for calculating is very simple. Below, we show some examples of visibility calculations in Dust 2. If you want to check positions in a local server, you can set `sv_cheats 1` and then type `getpos` in your game's console.

In [1]:
from awpy.vis import is_visible

ct_spawn_pos = (15, 2168, -65)
t_spawn_pos_1 = (-680, 834, 180)
t_spawn_pos_2 = (-1349, 814, 180)
mid_doors_ct = (-485.90, 1737.51, -60.28)
mid_doors_t = (-489.97, 1532.02, -61.08)
ct_spawn_towards_b = (-670.19, 2253.08, -56.78)
long_a_near_site = (1320.44, 2012.22, 61.44)

# Spawn positions should be visible to each other
print(
    f'T spawns visible from each other: {is_visible(t_spawn_pos_1, t_spawn_pos_2, "de_dust2")}'
)

# This is clearly not visible from each other
print(
    f'T spawn visible from CT spawn: {is_visible(t_spawn_pos_1, ct_spawn_pos, "de_dust2")}'
)

# A player cannot be visible if they're on the other side of the door!
print(
    f'Two sides of mid door visible: {is_visible(mid_doors_ct, mid_doors_t, "de_dust2")}'
)

# This is a long sightline, but it should still be visible
print(
    f'Long A visible from near B site: {is_visible(long_a_near_site, ct_spawn_towards_b, "de_dust2")}'
)

SyntaxError: EOL while scanning string literal (4133746492.py, line 15)